<h1 style="align: center;font-size: 18pt;">Multiprocess Lambdas</h1>
<hr style="height:2.5px">

This tutorial shows the user how to using the standard Python library `multiprocessing` to multiprocess lambda values in `BICePs`. 

<hr style="height:2.5px">

In [1]:
import numpy as np
import pandas as pd
import biceps
from biceps.decorators import multiprocess

BICePs - Bayesian Inference of Conformational Populations, Version 2.0


### Data and Output Directories

In [2]:
energies = np.loadtxt('cineromycin_B/cineromycinB_QMenergies.dat')*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
print(f"Possible input data extensions: {biceps.toolbox.list_possible_extensions()}")
input_data = biceps.toolbox.sort_data('cineromycin_B/J_NOE')
print(f"Input data: {biceps.toolbox.list_extensions(input_data)}")

Possible input data extensions: ['H', 'Ca', 'N', 'J', 'noe', 'pf']
Input data: ['J', 'noe']


### Parameters

In [4]:
nsteps=100000
n_lambdas = 2
nreplicas = 1
outdir = '%s_steps_%s_replica_%s_lam'%(nsteps, nreplicas, n_lambdas)
biceps.toolbox.mkdir(outdir)
print(f"nSteps of sampling: {nsteps}\nnReplicas: {nreplicas}")
lambda_values = np.linspace(0.0, 1.0, n_lambdas)
parameters = [dict(ref="uniform", sigma=(0.05, 20.0, 1.02)),
        dict(ref="exp", sigma=(0.05, 5.0, 1.02), gamma=(0.2, 5.0, 1.02)),]
pd.DataFrame(parameters)

nSteps of sampling: 100000
nReplicas: 1


,ref,sigma,gamma
0,uniform,"(0.05, 20.0, 1.02)",NaN
1,exp,"(0.05, 5.0, 1.02)","(0.2, 5.0, 1.02)"


In [5]:
@multiprocess(iterator=lambda_values)
def mp_lambdas(lam):
    print(f"lambda: {lam}")
    ensemble = biceps.Ensemble(lam, energies)
    ensemble.initialize_restraints(input_data, parameters)
    sampler = biceps.PosteriorSampler(ensemble.to_list(), nreplicas)
    sampler.sample(nsteps, print_freq=1000, verbose=False)
    sampler.traj.process_results(outdir+'/traj_lambda%2.2f.npz'%(lam))
    filename = outdir+'/sampler_lambda%2.2f.pkl'%(lam)
    biceps.toolbox.save_object(sampler, filename)

Number of CPU's: 6
Number of processes: 2
lambda: 0.0
lambda: 1.0

Accepted 63.987 % 


Accepted [ ...Nuisance paramters..., state] %
Accepted [32.747 29.703 29.703  1.537] % 


Accepted 70.721 % 


Accepted [ ...Nuisance paramters..., state] %
Accepted [32.546 29.753 29.753  8.422] % 



In [7]:
#%matplotlib inline
######## Posterior Analysis #######
#A = biceps.Analysis(outdir, nstates=len(energies))
#A.plot(show=True)

### Note that the output from each lambda will be returned to stdout as soon as the job has completed...

<h6 style="align: justify;font-size: 12pt"># <span style="color:red;">NOTE</span>: The following cell is for pretty notebook rendering</h6>

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../../theme.css", "r").read()
    return HTML(styles)
css_styling()